# COVID-19 — Interactive Map by Continent & Metric

## Intro
An interactive dashboard to explore COVID-19 cumulative metrics.  
Select a continent and a metric (cases, deaths, tests, vaccinations) to compare countries.

In [4]:
# --- Imports ---

import numpy as np
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# --- Load data ---
file_path = "../Data/Covid_19_dataset.csv"
df = pd.read_csv(file_path)

# Ensure datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# --- Project palette (consistent with previous notebooks) ---
project_palette = ["#0474ed","#91f0fa","#08a29e","#a2b458","#a6cabd","#326164"]

# for Plotly Express figures in Dash
palette_dash = project_palette


In [ ]:
# -- Cleaning (minimal)

df = df.copy()
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df[~df["iso_code"].str.startswith("OWID_")].copy()      # keep only real countries
df = df.sort_values(["iso_code", "date"])

# cumulative metrics to choose from
metric_cols = [
    "total_cases", "total_deaths", "total_tests",
    "total_vaccinations", "people_vaccinated"
]

# forward-fill within country, so the last day carries latest known totals
df[metric_cols] = df.groupby("iso_code")[metric_cols].ffill()

# -- Prepare last-day snapshot per country

last_rows = df.groupby("iso_code", as_index=False).tail(1)

countries = (
    last_rows[
        ["iso_code","continent","location","latitude","longitude"] + metric_cols
    ]
    .dropna(subset=["continent","latitude","longitude"])
    .reset_index(drop=True)
)

# labels for UI
METRIC_LABEL = {
    "total_cases": "Total cases",
    "total_deaths": "Total deaths",
    "total_tests": "Total tests",
    "total_vaccinations": "Total vaccinations",
    "people_vaccinated": "People vaccinated",
}

continents = sorted(countries["continent"].unique().tolist())
default_continent = "Europe" if "Europe" in continents else continents[0]
default_metric = "total_cases"

# --  Dash app

app = Dash(__name__)
app.title = "COVID-19 — Map by Continent & Metric"

app.layout = html.Div(
    style={"backgroundColor":"#ffffff", "padding":"12px", "fontFamily":"Arial, sans-serif"},
    children=[
        html.Div(style={"width":"1200px","height":"600px","margin":"0 auto"}, children=[
            html.H2("COVID-19 — Map by Continent & Metric", style={"margin":"0 0 8px 0"}),

            html.Div([
                html.Label("Select continent:", style={"fontWeight":600,"marginRight":"8px"}),
                dcc.Dropdown(
                    id="continent-dd",
                    options=[{"label":c,"value":c} for c in continents],
                    value=default_continent, clearable=False, style={"width":"320px","marginRight":"24px"}
                ),

                html.Label("Select metric:", style={"fontWeight":600,"marginRight":"8px"}),
                dcc.Dropdown(
                    id="metric-dd",
                    options=[{"label":lbl, "value":col} for col, lbl in METRIC_LABEL.items()],
                    value=default_metric, clearable=False, style={"width":"360px"}
                ),
            ], style={"display":"flex","alignItems":"center","marginBottom":"8px"}),

            dcc.Graph(id="map", style={"width":"100%","height":"100%"})
        ])
    ]
)


# -- Callback (returns only figure)

@app.callback(
    Output("map","figure"),
    Input("continent-dd","value"),
    Input("metric-dd","value")
)
def update_map(continent, metric):
    label = METRIC_LABEL.get(metric, metric)

    sub = countries[(countries["continent"] == continent) & (countries[metric].notna())].copy()
    if sub.empty:
        fig = px.scatter_map(lat=[], lon=[], zoom=1)
        fig.update_layout(mapbox_style="carto-darkmatter", paper_bgcolor="#ffffff", plot_bgcolor="#ffffff",
                          margin=dict(l=10,r=10,t=10,b=10), height=520, width=1200)
        return fig

    # size scaling (clip at 99th percentile to prevent extreme bubbles dominating)
    clip_u = float(np.nanquantile(sub[metric], 0.99))
    sub["marker_size"] = sub[metric].clip(upper=clip_u)

    # center the map on the selected continent
    center_lat = float(sub["latitude"].mean())
    center_lon = float(sub["longitude"].mean())

    fig = px.scatter_map(
        sub,
        lat="latitude", lon="longitude",
        size="marker_size", size_max=30,                 # half of 45
        color_discrete_sequence=palette_dash,
        hover_name="location",
        hover_data=None,                                 # keep it clean; we define tooltip below
        custom_data=[metric],                            # pass original metric value for tooltip
        zoom=2, center={"lat": center_lat, "lon": center_lon},
    )

    fig.update_traces(
        hovertemplate="<b>%{hovertext}</b><br>" + f"{label}: " + "%{customdata[0]:,}" + "<extra></extra>"
    )

    fig.update_layout(
        mapbox_style="carto-darkmatter",
        paper_bgcolor="#ffffff", plot_bgcolor="#ffffff",
        margin=dict(l=10,r=10,t=10,b=10),
        height=520,
        width=1200
    )
    return fig

## Explanation
- **Bubble size** is proportional to the selected metric.  
- Extreme outliers are clipped (99th percentile) to keep the map readable.  
- Values are forward-filled per country, so the last available day has the latest totals.

In [29]:
import socket
def get_free_port():
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

free_port = get_free_port()
app.run(debug=True, use_reloader=False, port=free_port)

## Summary
The map clearly shows regional differences in the pandemic.  
Europe and the Americas stand out with the largest bubbles, reflecting high cumulative cases and vaccinations.  
This visualization highlights how unevenly the impact of COVID-19 was distributed across continents.